In [1]:
import sqlite3
import pandas as pd


db_name = 'data.db'

conn = sqlite3.connect(db_name)
cursor = conn.cursor()

# Crina a tabela

cursor.execute('''
CREATE TABLE IF NOT EXISTS vendas (
        ID INTEGER PRIMARY KEY,
        Produto TEXT,
        Categoria TEXT,
        Preço REAL,
        Quantidade_Vendida INTEGER,
        Data_Venda TEXT,
        Loja TEXT,
        Vendedor TEXT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS estoque (
        ID INTEGER PRIMARY KEY,
        Produto TEXT,
        Categoria TEXT,
        Estoque_Inicial INTEGER,
        Estoque_Atual INTEGER,
        Fornecedor TEXT,
        Data_Reabastecimento TEXT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS estoque_detalhado (
        ID INTEGER PRIMARY KEY,
        Código_SKU INTERGER,
        Código_Barras INTERGER,
        Ultima_Compra TEXT,
        Quantidade_Recebida INTEGER,
        Data_Proxima_Entrega TEXT,
        Data_Ultima_Saida TEXT,
        Quantidade_Saida INTEGER,
        Status_Estoque TEXT,
        Temperatura_Armazenamento TEXT
    )
''')

df_vendas = pd.read_csv('./dataset/Base_de_Vendas.csv')
df_estoque = pd.read_csv('./dataset/Base_de_Estoque.csv')
df_estoque_2 = pd.read_csv('./dataset/base_estoque_detalhado.csv')

# Inserindo dados no banco de dados

df_vendas.to_sql('vendas', conn, if_exists='append', index=False)
df_estoque.to_sql('estoque', conn, if_exists='append', index=False)
df_estoque_2.to_sql('estoque_detalhado', conn, if_exists='append', index=False)

# Commit e fechar conexão
conn.commit()
conn.close()

print('Dados inseridos com sucesso!')



Dados inseridos com sucesso!


In [2]:
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

# Exibindo os dados

print('Amostra da tabela de Vendas:')
for row in cursor.execute('SELECT * FROM vendas LIMIT 5'):
    print(row)
    
print('\nAmostra da tabela de Estoque:')
for row in cursor.execute('SELECT * FROM estoque LIMIT 5'):
    print(row)

print('\nAmostra da tabela de Estoque Detalhado:')
for row in cursor.execute('SELECT * FROM estoque_detalhado LIMIT 5'):
    print(row)
    
conn.close()

Amostra da tabela de Vendas:
(1, 'Produto_1', 'Móveis', 391.84, 42, '2023-01-01', 'Loja E', 'Vendedor_8')
(2, 'Produto_2', 'Brinquedos', 84.0, 13, '2023-01-02', 'Loja E', 'Vendedor_15')
(3, 'Produto_3', 'Móveis', 306.32, 35, '2023-01-03', 'Loja C', 'Vendedor_19')
(4, 'Produto_4', 'Alimentos', 124.38, 32, '2023-01-04', 'Loja A', 'Vendedor_17')
(5, 'Produto_5', 'Móveis', 378.73, 23, '2023-01-05', 'Loja B', 'Vendedor_8')

Amostra da tabela de Estoque:
(1, 'Produto_1', 'Móveis', 264, 222, 'Fornecedor_7', '2023-01-01')
(2, 'Produto_2', 'Brinquedos', 445, 432, 'Fornecedor_10', '2023-01-16')
(3, 'Produto_3', 'Móveis', 213, 178, 'Fornecedor_8', '2023-01-31')
(4, 'Produto_4', 'Alimentos', 407, 375, 'Fornecedor_4', '2023-02-15')
(5, 'Produto_5', 'Móveis', 431, 408, 'Fornecedor_8', '2023-03-02')

Amostra da tabela de Estoque Detalhado:
(1, 'SKU70430', 235625627231, '2023-01-01', 143, '2023-07-01', '2023-02-01', 190, 'Esgotado', 'Refrigerado')
(2, 'SKU45360', 699068686642, '2023-01-16', 149, '2023

In [3]:
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

query = '''
    SELECT v.ID, v.Produto, v.Categoria, v.Preço, v.Quantidade_Vendida, v.Data_Venda, e.Estoque_Atual, e.Fornecedor, ed.Código_SKU, ed.Código_Barras
    FROM vendas v
    JOIN estoque e ON v.ID = e.ID
    join estoque_detalhado ed ON e.ID = ed.ID
'''

df_join = pd.read_sql_query(query, conn)
conn.close()

print(df_join.head())

   ID    Produto   Categoria   Preço  Quantidade_Vendida  Data_Venda  \
0   1  Produto_1      Móveis  391.84                  42  2023-01-01   
1   2  Produto_2  Brinquedos   84.00                  13  2023-01-02   
2   3  Produto_3      Móveis  306.32                  35  2023-01-03   
3   4  Produto_4   Alimentos  124.38                  32  2023-01-04   
4   5  Produto_5      Móveis  378.73                  23  2023-01-05   

   Estoque_Atual     Fornecedor Código_SKU  Código_Barras  
0            222   Fornecedor_7   SKU70430   235625627231  
1            432  Fornecedor_10   SKU45360   699068686642  
2            178   Fornecedor_8   SKU87820   467137320835  
3            375   Fornecedor_4   SKU72528   750280489504  
4            408   Fornecedor_8   SKU15662   123148787110  
